In [49]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from uk_covid19 import Cov19API
# import module

In [50]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [51]:
def parse_date(datestring):#convert a data sting
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [115]:
def Access_API_England():
    filters_England = ['areaType=nation','areaName=England']
    structure_testbypillar_plancap = { 'date':'date',
               'pillarone':'cumPillarOneTestsByPublishDate',
               'pillartwo':'cumPillarTwoTestsByPublishDate',
               'pillarthree':'cumPillarThreeTestsByPublishDate',
               'pillarfour':'cumPillarFourTestsByPublishDate'}
    api_E = Cov19API(filters=filters_England, structure=structure_testbypillar_plancap)
    TESTPILLAR_E=api_E.get_json()
    with open("TESTPILLAR_E.json", "wt") as A_E:
        json.dump(TESTPILLAR_E,  A_E)
       

In [116]:
def Access_API_Scotland():
    filters_Scotland = ['areaType=nation','areaName=Scotland']
    structure_testbypillar_plancap = { 'date':'date',
               'pillarone':'cumPillarOneTestsByPublishDate',
               'pillartwo':'cumPillarTwoTestsByPublishDate',
               'pillarthree':'cumPillarThreeTestsByPublishDate',
               'pillarfour':'cumPillarFourTestsByPublishDate'}
    api_S = Cov19API(filters=filters_Scotland, structure=structure_testbypillar_plancap)
    TESTPILLAR_S=api_S.get_json()
    with open("TESTPILLAR_S.json", "wt") as A_S:
        json.dump(TESTPILLAR_S,  A_S)
    

In [117]:
def Access_API_Wale():
    filters_Wales = ['areaType=nation','areaName=Wales']
    structure_testbypillar_plancap = { 'date':'date',
               'pillarone':'cumPillarOneTestsByPublishDate',
               'pillartwo':'cumPillarTwoTestsByPublishDate',
               'pillarthree':'cumPillarThreeTestsByPublishDate',
               'pillarfour':'cumPillarFourTestsByPublishDate'}
    api_W = Cov19API(filters=filters_Wales, structure=structure_testbypillar_plancap)
    TESTPILLAR_W=api_W.get_json()
    with open("TESTPILLAR_W.json", "wt") as A_W:
        json.dump(TESTPILLAR_W,  A_W)
     

In [136]:
def Access_API_NorthernIreland():
    filters_NorthernIreland = ['areaType=nation','areaName=Northern Ireland']
    structure_testbypillar_plancap = { 'date':'date',
               'pillarone':'cumPillarOneTestsByPublishDate',
               'pillartwo':'cumPillarTwoTestsByPublishDate',
               'pillarthree':'cumPillarThreeTestsByPublishDate',
               'pillarfour':'cumPillarFourTestsByPublishDate'}
    api_NI = Cov19API(filters=filters_NorthernIreland, structure=structure_testbypillar_plancap)
    TESTPILLAR_NI=api_NI.get_json()
    with open("TESTPILLAR_NI.json", "wt") as A_NI:
        json.dump(TESTPILLAR_NI,  A_NI)
    

In [137]:
with open("TESTPILLAR_E.json", "rt") as PILLAR_E:
    data_E=json.load(PILLAR_E)
with open("TESTPILLAR_S.json", "rt") as PILLAR_S:
    data_S=json.load(PILLAR_S)
with open("TESTPILLAR_W.json", "rt") as PILLAR_W:
    data_W=json.load(PILLAR_W)
with open("TESTPILLAR_NI.json", "rt") as PILLAR_NI:
    data_NI=json.load(PILLAR_NI)

In [138]:
def wrangle_data(data):
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['pillarone', 'pillartwo', 'pillarthree','pillarfour'])
    
    for entry in datalist: # each entry is a dictionary with pillarone, pillartwo, pillarthree and pillarfour
        date=parse_date(entry['date'])
        for column in ['pillarone', 'pillartwo', 'pillarthree','pillarfour']:
            if pd.isna(timeseriesdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.1
                timeseriesdf.loc[date, column]=value
    timeseriesdf.fillna(0.0, inplace=True)
    timeseriesdf.to_pickle("timeseriesdf.pkl")

In [149]:
wrangle_data(data_E)
timeseriesdfE=pd.read_pickle("timeseriesdf.pkl")
wrangle_data(data_S)
timeseriesdfS=pd.read_pickle("timeseriesdf.pkl")
wrangle_data(data_W)
timeseriesdfW=pd.read_pickle("timeseriesdf.pkl")
wrangle_data(data_NI)
timeseriesdfNI=pd.read_pickle("timeseriesdf.pkl")
# Create a widget for the user to select columns
series=wdg.SelectMultiple(
    options=['pillarone', 'pillartwo', 'pillarthree','pillarfour'],
    value=['pillarone', 'pillartwo', 'pillarthree','pillarfour'],
    rows=4,
    description='Stats:',
    disabled=False
)
# creat a widget for the use to select region
region = wdg.RadioButtons(
    options=['England', 'Wale', 'Scotland','Northern Ireland'],
    description='Region:',
    disabled=False
)

# Create a button to allow users to flick between log and linear displays
scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)


# Create a button to allow users to refresh the data
ni_apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='primary', 
    tooltip="Click to refresh the data in the graph",
    icon='refresh'
)

# try replacing HBox with a VBox

def timeseries_graph(gcols, gscale, gregion):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0 and gregion == 'England':
              timeseriesdfE[list(gcols)].plot(logy=logscale)
    if ncols>0 and gregion =='Wale':
              timeseriesdfW[list(gcols)].plot(logy=logscale)
    if ncols>0 and gregion =='Scotland':
              timeseriesdfS[list(gcols)].plot(logy=logscale)
    if ncols>0 and gregion =='Northern Ireland':
              timeseriesdfNI[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale,'gregion':region})

##################################################################################################
def refresh():
    current=series.value
    series.value=() # forces the redraw
    series.value=current # now we can change it back
    timeseriesbutton.button_style='success'
    timeseriesbutton.description='Successful'
    time.sleep(3)
    timeseriesbutton.button_style=''
    timeseriesbutton.description='Donwload timeseries data'
def api_button_callback(button):
        """ Button callback - it must take the button as its para to accesses API, wrangles data, updates global variable df used for plotting. """
        # Try calling the api
        try:

            timeseriesdata_uk=timeseries_uk()
            timeseriesdata_england=timeseries_england()
            timeseriesdata_scotland=timeseries_scotland()
            timeseriesdata_wales=timeseries_wales()
            refresh()
            # Update the Button to show the API was successful
            apibutton.description='Successful!'
            apibutton.icon="check"
            apibutton.button_style='success'
            apibutton.disabled=True
            time.sleep(3)
            # Reset the Button 
            apibutton.description='Refresh Data'
            apibutton.icon="refresh"
            apibutton.button_style='primary'
            apibutton.disabled=False
        except:
            # Update the Button to show the API failed
            apibutton.description='Error'
            apibutton.icon="fa-exclamation-triangle"
            apibutton.button_style='warning'
            apibutton.disabled=True
            time.sleep(3)
            # Reset the Button
            apibutton.description='Refresh Data'
            apibutton.icon="refresh"
            apibutton.button_style='primary'
            apibutton.disabled=False
apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='primary', 
    tooltip="Click to refresh the data in the graph",
    icon='refresh'
)
apibutton.on_click(api_button_callback)
display(apibutton)
####################################################################################################
# stack series and scale on top of each other
ctrls=wdg.VBox([series, scale,region])
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])
display(form)

Button(button_style='primary', description='Refresh Data', icon='refresh', style=ButtonStyle(), tooltip='Click…